In [4]:
# import pandas as pd
import numpy as np, pandas as pd
import matplotlib.pyplot as plt
from matplotlib import pyplot
import seaborn as sns
from statsmodels.graphics.gofplots import qqplot
from scipy.stats import shapiro
from scipy.stats import normaltest
from datetime import datetime

# Read the 3 indices first

nq5m_df = pd.read_csv("C:\\Users\\Shuv\\Documents\\All Trading\\Market Data\\Research Data\\CME_MINI_NQ_5Min_18Mar22.csv")
#es5m_df = pd.read_csv("C:\\Users\\Shuv\\Documents\\All Trading\\Market Data\\Research Data\\CME_MINI_ES_5Min_18Mar22.csv")
#ym5m_df = pd.read_csv("C:\\Users\\Shuv\\Documents\\All Trading\\Market Data\\Research Data\\CME_MINI_YM_5Min_18Mar22.csv")

nq5m_df.set_index('time')
#es5m_df.set_index('time')
#ym5m_df.set_index('time')

,open,high,low,close,Developing Poc,Developing VA High,Developing VA Low,EMA200,Smoothing Line,EMA50,...,ATR,CDV Line,CDV Candles (Open),CDV Candles (High,CDV Candles (Low),CDV Candles (Close),SMA 1,SMA 2,EMA 1,EMA 2
time,,,,,,,,,,,,,,,,,,,,,
2022-03-10T18:20:00Z,13467.75,13476.50,13452.25,13463.75,13531.375,13593.25,13422.75,13565.81296,13568.01959,13475.47971,...,29.502570,NaN,-225194.97790,-222608.17980,-225194.97790,-223249.19270,NaN,NaN,NaN,NaN
2022-03-10T18:25:00Z,13465.50,13482.75,13451.25,13474.25,13531.375,13593.25,13422.75,13564.90188,13566.90049,13475.43149,...,29.645243,NaN,-224222.08530,-222459.73340,-224222.08530,-223174.96950,NaN,NaN,NaN,NaN
2022-03-10T18:30:00Z,13473.50,13497.75,13463.00,13490.00,13531.375,13593.25,13422.75,13564.15659,13565.90844,13476.00280,...,30.009869,NaN,-223698.52740,-220195.88440,-223698.52740,-221327.80890,NaN,NaN,NaN,NaN
2022-03-10T18:35:00Z,13489.75,13504.25,13478.00,13483.25,13531.375,13590.50,13422.75,13563.35155,13565.01234,13476.28701,...,29.741307,NaN,-222513.16810,-220195.88440,-222513.16810,-221094.48210,NaN,NaN,NaN,NaN
2022-03-10T18:40:00Z,13484.00,13492.25,13465.00,13470.25,13531.375,13590.50,13422.75,13562.42517,13564.12963,13476.05026,...,29.563356,NaN,-221803.82510,-221803.82510,-223851.99710,-222922.53840,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-03-18T20:35:00Z,14433.00,14435.25,14430.25,14433.00,14301.375,14352.25,14022.25,14176.64541,14171.43198,14344.10583,...,18.582481,NaN,-41187.22521,-40657.94997,-41187.22521,-40745.44997,NaN,NaN,NaN,NaN
2022-03-18T20:40:00Z,14432.75,14436.75,14431.75,14432.50,14301.375,14352.25,14022.25,14179.19123,14174.04360,14347.57227,...,17.612303,NaN,-40966.33759,-40657.94997,-40966.33759,-40760.58747,NaN,NaN,NaN,NaN
2022-03-18T20:45:00Z,14433.50,14434.75,14426.00,14430.25,14301.375,14352.25,14022.25,14181.68933,14176.61233,14350.81453,...,16.979282,NaN,-40863.46253,-40863.22497,-41310.31068,-41086.76782,NaN,NaN,NaN,NaN


In [5]:
from sklearn import linear_model
from sklearn.preprocessing import StandardScaler
from statsmodels.api import OLS


# Drop the columns with Nan values
nq5m_df= nq5m_df.dropna(axis=1)
#Construct the Input matrix, with all columns except 'close' which is the Y
#X = All Input columns except 'close' & 'time' upto the 2nd last row


#X = pd.DataFrame(data= (nq5m_df.loc[:,[col for col in nq5m_df.columns if col!='close' and col!='time'] ]).diff()[1:-1])
X = pd.DataFrame(data= (nq5m_df.loc[:,[col for col in nq5m_df.columns if col!='close' and col!='time'] ])[:-1])
scaler = StandardScaler()
scaled_X = scaler.fit_transform(X)

scaled_X_df = pd.DataFrame(scaled_X, columns = X.columns)
#Y = 'close' diff
Y = nq5m_df['close'].diff()[1:]
#Convert into right 2D shape to do regression
Y_df = Y.values.reshape(-1,1)
#Do the OLS regression now
#def calc_phi(close,i):
 #   return (close.iloc[i]*close.iloc[i-1])/close
        
#df['CustomRating'] = df.apply(lambda x: custom_rating(x['Genre'],x['Rating']),axis=1)

model = OLS(Y_df,scaled_X_df)
results = model.fit()
print(results.summary())

                                 OLS Regression Results                                
Dep. Variable:                      y   R-squared (uncentered):                   0.025
Model:                            OLS   Adj. R-squared (uncentered):              0.013
Method:                 Least Squares   F-statistic:                              1.981
Date:                Wed, 30 Mar 2022   Prob (F-statistic):                     0.00436
Time:                        02:09:03   Log-Likelihood:                         -7139.8
No. Observations:                1699   AIC:                                  1.432e+04
Df Residuals:                    1677   BIC:                                  1.444e+04
Df Model:                          22                                                  
Covariance Type:            nonrobust                                                  
                          coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------